# "backtesting crypto"
> "How to fetch and backtest crypto data using fastquant"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Jerome de Leon
- categories: [crypto, backtest, grid search]

In [1]:
# uncomment to install in colab
# !pip3 install fastquant

## fetch data from binance

In [2]:
from fastquant import get_crypto_data

crypto = get_crypto_data("BTC/USDT", 
                         "2018-12-01", 
                         "2019-12-31",
                         time_resolution='1d'
                        )

In [3]:
crypto

,open,high,low,close,volume
dt,,,,,
2018-12-01,4041.27,4299.99,3963.01,4190.02,44840.073481
2018-12-02,4190.98,4312.99,4103.04,4161.01,38912.154790
2018-12-03,4160.55,4179.00,3827.00,3884.01,49094.369163
2018-12-04,3884.76,4085.00,3781.00,3951.64,48489.551613
2018-12-05,3950.98,3970.00,3745.00,3769.84,44004.799448
...,...,...,...,...,...
2019-12-27,7202.00,7275.86,7076.42,7254.74,33642.701861
2019-12-28,7254.77,7365.01,7238.67,7316.14,26848.982199
2019-12-29,7315.36,7528.45,7288.00,7388.24,31387.106085


## run backtest with a grid of values

In [4]:
from fastquant import backtest

results = backtest('smac', 
                   crypto, 
                   fast_period=[7,14,21,28], 
                   slow_period=[30,45,60,75],
                   plot=False,
                   verbose=False
                  )

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 30
Final Portfolio Value: 167957.05730000004
Final PnL: 67957.06
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 45
Final Portfolio Value: 200109.894525
Final PnL: 100109.89
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 60
Final Portfolio Value: 189298.80590000006
Final PnL: 89298.81
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 75
Final Portfolio Value: 258316.23405000006
Final PnL: 158316.23
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level argume

In [5]:
results.head()

,init_cash,buy_prop,sell_prop,commission,execution_type,fast_period,slow_period,rtot,ravg,rnorm,rnorm100,sharperatio,pnl,final_value
0,100000,1,1,0.0075,close,7,75,0.949014,0.002397,0.829272,82.927229,0.987367,158316.23,258316.234050
1,100000,1,1,0.0075,close,28,60,0.931504,0.002352,0.809002,80.900205,0.986999,153832.42,253832.420400
2,100000,1,1,0.0075,close,14,75,0.830975,0.002098,0.696898,69.689847,0.984563,129555.54,229555.539175
3,100000,1,1,0.0075,close,21,75,0.782243,0.001975,0.645083,64.508323,0.983142,118637.07,218637.072700
4,100000,1,1,0.0075,close,28,75,0.769574,0.001943,0.631874,63.187426,0.982741,115884.74,215884.739100


That's a 258% maximum profit using only SMAC because bitcoin was bullish all time long!

In [6]:
#get best parameters on top row 
fast_best, slow_best = results.iloc[0][["fast_period","slow_period"]]
fast_best, slow_best

(7, 75)

## run backtest using optimum values

In [7]:
import matplotlib as pl
pl.style.use("default")
pl.rcParams["figure.figsize"] = (9,5)

In [8]:
results = backtest('smac', 
                   crypto, 
                   fast_period=fast_best, 
                   slow_period=slow_best,
                   plot=True,
                   verbose=False,
                   return_transactions=True
                  )

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 75
2019-02-14, BUY CREATE, 3590.56
2019-02-14, Cash: 100000.0
2019-02-14, Price: 3590.56
2019-02-14, Buy prop size: 27
2019-02-14, Afforded size: 27
2019-02-14, Final size: 27
2019-02-15, BUY EXECUTED, Price: 3590.56, Cost: 96945.12, Comm: 727.09
2019-08-20, SELL CREATE, 10142.57
2019-08-21, SELL EXECUTED, Price: 10142.57, Cost: 96945.12, Comm: 2053.87
2019-08-21, OPERATION PROFIT, GROSS 176904.27, NET 174123.31
2019-11-01, BUY CREATE, 9231.61
2019-11-01, Cash: 274123.31117500004
2019-11-01, Price: 9231.61
2019-11-01, Buy prop size: 29
2019-11-01, Afforded size: 29
2019-11-01, Final size: 29
2019-11-02, BUY EXECUTED, Price: 9231.61, Cost: 267716.69, Comm: 2007.88
2019-11-11, SELL CREATE, 8821.94
2019-11-12, SELL EXECUTED, Price: 8821.94, Cost: 267716.69, Comm: 1918.77
2019-11-12, OPERATION PROFIT, GROSS -11880.43, NET -15807.08
Fina

## accessing transactions history

This is done by setting `return_transactions`=True

### single strategy

In [9]:
results, transactions = backtest('smac', 
                               crypto, 
                               fast_period=fast_best, 
                               slow_period=slow_best,
                               plot=False,
                               verbose=False,
                               return_transactions=True
                              )

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 75
2019-02-14, BUY CREATE, 3590.56
2019-02-14, Cash: 100000.0
2019-02-14, Price: 3590.56
2019-02-14, Buy prop size: 27
2019-02-14, Afforded size: 27
2019-02-14, Final size: 27
2019-02-15, BUY EXECUTED, Price: 3590.56, Cost: 96945.12, Comm: 727.09
2019-08-20, SELL CREATE, 10142.57
2019-08-21, SELL EXECUTED, Price: 10142.57, Cost: 96945.12, Comm: 2053.87
2019-08-21, OPERATION PROFIT, GROSS 176904.27, NET 174123.31
2019-11-01, BUY CREATE, 9231.61
2019-11-01, Cash: 274123.31117500004
2019-11-01, Price: 9231.61
2019-11-01, Buy prop size: 29
2019-11-01, Afforded size: 29
2019-11-01, Final size: 29
2019-11-02, BUY EXECUTED, Price: 9231.61, Cost: 267716.69, Comm: 2007.88
2019-11-11, SELL CREATE, 8821.94
2019-11-12, SELL EXECUTED, Price: 8821.94, Cost: 267716.69, Comm: 1918.77
2019-11-12, OPERATION PROFIT, GROSS -11880.43, NET -15807.08
Fina

In [10]:
transactions.keys()

dict_keys(['smac'])

In [11]:
transactions

{'smac':            buy_price sell_price       cost commision
 dt                                                  
 2019-02-15   3590.56       None   96945.12    727.09
 2019-08-21      None   10142.57   96945.12   2053.87
 2019-11-02   9231.61       None  267716.69   2007.88
 2019-11-12      None    8821.94  267716.69   1918.77}

### multi strategy

In [12]:
from fastquant import backtest

results, transactions = backtest('multi', 
                                   crypto, 
                                   strats= {
                                       'smac': {
                                           'fast_period': [7,14,21], 
                                           'slow_period': [30,45,60]
                                            },
                                        'rsi': {
                                            'rsi_upper': [70,80],
                                            'rsi_lower': [20,30] 
                                        }
                                   },
                                   plot=False,
                                   verbose=False,
                                   return_transactions=True
                                  )

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 30
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 70
rsi_lower : 20
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop size: 30
2018-12-15, Afforded size: 30
2018-12-15, Final size: 30
2018-12-16, BUY EXECUTED, Price: 3211.72, Cost: 96351.60, Comm: 722.64
2019-01-13, SELL CREATE, 3626.09
2019-01-14, SELL EXECUTED, Price: 3626.09, Cost: 96351.60, Comm: 815.87
2019-02-11, BUY CREATE, 3631.05
2019-02-11, Cash: 110892.59275000001
2019-02-11, Price: 3631.05
2019-02-11, Buy prop size: 30
2019-02-11, Afforded size: 30
2019-02-11, Final size: 30
2019-02-12, BUY EXECUTED, Price: 3631.05, Cost: 108931.50, Comm: 816.99
2019-02-12, OPERATION PROFIT, GROSS -148.80, NET -1781.66
2019-02

2019-09-27, BUY EXECUTED, Price: 8063.73, Cost: 161274.60, Comm: 1209.56
2019-11-16, SELL CREATE, 8502.40
2019-11-17, SELL EXECUTED, Price: 8502.40, Cost: 161274.60, Comm: 1275.36
2019-12-28, BUY CREATE, 7316.14
2019-12-28, Cash: 169405.39240000004
2019-12-28, Price: 7316.14
2019-12-28, Buy prop size: 22
2019-12-28, Afforded size: 22
2019-12-28, Final size: 22
2019-12-29, BUY EXECUTED, Price: 7316.14, Cost: 160955.08, Comm: 1207.16
Final Portfolio Value: 165538.20930000002
Final PnL: 65538.21
Final Portfolio Value: 165538.20930000002
Final PnL: 65538.21
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 7
slow_period : 30
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 80
rsi_lower : 30
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop si

2019-08-07, BUY CREATE, 11975.03
2019-08-07, Cash: 149793.928
2019-08-07, Price: 11975.03
2019-08-07, Buy prop size: 12
2019-08-07, Afforded size: 12
2019-08-07, Final size: 12
2019-08-08, BUY EXECUTED, Price: 11975.03, Cost: 143700.36, Comm: 1077.75
2019-08-17, SELL CREATE, 10306.78
2019-08-18, SELL EXECUTED, Price: 10306.78, Cost: 143700.36, Comm: 927.61
2019-08-18, OPERATION PROFIT, GROSS -20019.00, NET -22024.36
2019-09-08, BUY CREATE, 10381.18
2019-09-08, Cash: 127769.56510000002
2019-09-08, Price: 10381.18
2019-09-08, Buy prop size: 12
2019-09-08, Afforded size: 12
2019-09-08, Final size: 12
2019-09-09, BUY EXECUTED, Price: 10381.18, Cost: 124574.16, Comm: 934.31
2019-09-09, SELL CREATE, 10098.15
2019-09-10, SELL EXECUTED, Price: 10098.15, Cost: 124574.16, Comm: 908.83
2019-09-10, OPERATION PROFIT, GROSS -3396.36, NET -5239.50
2019-09-26, BUY CREATE, 8063.73
2019-09-26, Cash: 122530.0654
2019-09-26, Price: 8063.73
2019-09-26, Buy prop size: 15
2019-09-26, Afforded size: 15
2019-0

2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop size: 30
2018-12-15, Afforded size: 30
2018-12-15, Final size: 30
2018-12-16, BUY EXECUTED, Price: 3211.72, Cost: 96351.60, Comm: 722.64
2019-04-02, SELL CREATE, 4932.60
2019-04-03, SELL EXECUTED, Price: 4932.60, Cost: 96351.60, Comm: 1109.84
2019-04-03, OPERATION PROFIT, GROSS 51626.40, NET 49793.93
2019-08-04, BUY CREATE, 10929.23
2019-08-04, Cash: 149793.928
2019-08-04, Price: 10929.23
2019-08-04, Buy prop size: 13
2019-08-04, Afforded size: 13
2019-08-04, Final size: 13
2019-08-05, BUY EXECUTED, Price: 10929.23, Cost: 142079.99, Comm: 1065.60
2019-08-17, SELL CREATE, 10306.78
2019-08-18, SELL EXECUTED, Price: 10306.78, Cost: 142079.99, Comm: 1004.91
2019-08-18, OPERATION PROFIT, GROSS -8091.85, NET -10162.36
2019-09-08, BUY CREATE, 10381.18
2019-09-08, Cash: 139631.56702500005
2019-09-08, Price: 10381.18
2019-09-08, Buy prop size: 13
2019-09-08, Afforded size: 13
2019-09-08, F

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 14
slow_period : 30
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 80
rsi_lower : 30
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop size: 30
2018-12-15, Afforded size: 30
2018-12-15, Final size: 30
2018-12-16, BUY EXECUTED, Price: 3211.72, Cost: 96351.60, Comm: 722.64
2019-01-15, SELL CREATE, 3591.84
2019-01-16, SELL EXECUTED, Price: 3591.84, Cost: 96351.60, Comm: 808.16
2019-02-14, BUY CREATE, 3590.56
2019-02-14, Cash: 109872.799
2019-02-14, Price: 3590.56
2019-02-14, Buy prop size: 30
2019-02-14, Afforded size: 30
2019-02-14, Final size: 30
2019-02-15, BUY EXECUTED, Price: 3590.56, Cost: 107716.80, Comm: 807.88
2019-02-15, OPERATION PROFIT, GROSS 38.40, NET -1577.64
2019-04-02, SELL

2019-01-21, SELL CREATE, 3570.93
2019-01-22, SELL EXECUTED, Price: 3570.93, Cost: 96351.60, Comm: 803.46
2019-02-18, BUY CREATE, 3898.60
2019-02-18, Cash: 109250.20375
2019-02-18, Price: 3898.6
2019-02-18, Buy prop size: 27
2019-02-18, Afforded size: 27
2019-02-18, Final size: 27
2019-02-19, BUY EXECUTED, Price: 3898.60, Cost: 105262.20, Comm: 789.47
2019-04-02, SELL CREATE, 4932.60
2019-04-03, SELL EXECUTED, Price: 4932.60, Cost: 105262.20, Comm: 998.85
2019-08-11, BUY CREATE, 11549.97
2019-08-11, Cash: 135379.88575000002
2019-08-11, Price: 11549.97
2019-08-11, Buy prop size: 11
2019-08-11, Afforded size: 11
2019-08-11, Final size: 11
2019-08-12, BUY EXECUTED, Price: 11549.97, Cost: 127049.67, Comm: 952.87
2019-08-24, SELL CREATE, 10142.69
2019-08-25, SELL EXECUTED, Price: 10142.69, Cost: 127049.67, Comm: 836.77
2019-09-24, BUY CREATE, 8493.14
2019-09-24, Cash: 118110.16130000004
2019-09-24, Price: 8493.14
2019-09-24, Buy prop size: 13
2019-09-24, Afforded size: 13
2019-09-24, Final s

2019-09-24, Buy prop size: 14
2019-09-24, Afforded size: 14
2019-09-24, Final size: 14
2019-09-25, BUY EXECUTED, Price: 8493.14, Cost: 118903.96, Comm: 891.78
2019-11-21, SELL CREATE, 7268.23
2019-11-22, SELL EXECUTED, Price: 7268.23, Cost: 118903.96, Comm: 763.16
2019-11-22, BUY CREATE, 7268.23
2019-11-22, Cash: 106202.59565
2019-11-22, Price: 7268.23
2019-11-22, Buy prop size: 14
2019-11-22, Afforded size: 14
2019-11-22, Final size: 14
2019-11-23, BUY EXECUTED, Price: 7268.23, Cost: 101755.22, Comm: 763.16
Final Portfolio Value: 104417.4315
Final PnL: 4417.43
Final Portfolio Value: 104417.4315
Final PnL: 4417.43
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 21
slow_period : 30
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 70
rsi_lower : 20
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Ca

2019-11-21, SELL CREATE, 7268.23
2019-11-22, SELL EXECUTED, Price: 7268.23, Cost: 85769.37, Comm: 490.61
Final Portfolio Value: 65411.081425000004
Final PnL: -34588.92
Final Portfolio Value: 65411.081425000004
Final PnL: -34588.92
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 21
slow_period : 30
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 80
rsi_lower : 30
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop size: 30
2018-12-15, Afforded size: 30
2018-12-15, Final size: 30
2018-12-16, BUY EXECUTED, Price: 3211.72, Cost: 96351.60, Comm: 722.64
2019-01-14, SELL CREATE, 3553.06
2019-01-15, SELL EXECUTED, Price: 3553.06, Cost: 96351.60, Comm: 799.44
2019-02-18, BUY CREATE, 3898.60
2019-02-18, Cash: 108718.12449999999
2019-02-18, Price: 

2019-08-17, BUY CREATE, 10216.02
2019-08-17, Cash: 133275.57790000003
2019-08-17, Price: 10216.02
2019-08-17, Buy prop size: 12
2019-08-17, Afforded size: 12
2019-08-17, Final size: 12
2019-08-18, BUY EXECUTED, Price: 10216.02, Cost: 122592.24, Comm: 919.44
2019-08-30, SELL CREATE, 9587.47
2019-08-31, SELL EXECUTED, Price: 9587.47, Cost: 122592.24, Comm: 862.87
2019-09-26, BUY CREATE, 8063.73
2019-09-26, Cash: 123950.66380000001
2019-09-26, Price: 8063.73
2019-09-26, Buy prop size: 15
2019-09-26, Afforded size: 15
2019-09-26, Final size: 15
2019-09-27, BUY EXECUTED, Price: 8063.73, Cost: 120955.95, Comm: 907.17
2019-11-24, SELL CREATE, 7109.57
2019-11-25, SELL EXECUTED, Price: 7109.57, Cost: 120955.95, Comm: 799.83
Final Portfolio Value: 107931.26755000002
Final PnL: 7931.27
Final Portfolio Value: 107931.26755000002
Final PnL: 7931.27
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 21
slow_perio

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 21
slow_period : 60
===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
rsi_period : 14
rsi_upper : 80
rsi_lower : 30
2018-12-15, BUY CREATE, 3211.72
2018-12-15, Cash: 100000.0
2018-12-15, Price: 3211.72
2018-12-15, Buy prop size: 30
2018-12-15, Afforded size: 30
2018-12-15, Final size: 30
2018-12-16, BUY EXECUTED, Price: 3211.72, Cost: 96351.60, Comm: 722.64
2019-04-02, SELL CREATE, 4932.60
2019-04-03, SELL EXECUTED, Price: 4932.60, Cost: 96351.60, Comm: 1109.84
2019-04-03, OPERATION PROFIT, GROSS 51626.40, NET 49793.93
2019-08-19, BUY CREATE, 10915.54
2019-08-19, Cash: 149793.928
2019-08-19, Price: 10915.54
2019-08-19, Buy prop size: 13
2019-08-19, Afforded size: 13
2019-08-19, Final size: 13
2019-08-20, BUY EXECUTED, Price: 10915.54, Cost: 141902.02, Comm: 1064.27
2019-08-

In [13]:
transactions.keys()

dict_keys(['smac_slow_period30', 'rsi_rsi_lower20', 'rsi_rsi_lower30', 'smac_slow_period45', 'smac_slow_period60'])

In [14]:
transactions

{'smac_slow_period30':            buy_price sell_price       cost commision
 dt                                                  
 2019-01-15      None    3553.06   96351.60    799.44
 2019-02-19   3898.60       None  105262.20    789.47
 2019-03-17      None    3981.14  105262.20    806.18
 2019-03-22   3980.64       None  107477.28    806.08
 2019-08-12  11549.97       None  127049.67    952.87
 2019-08-29      None    9498.44  127049.67    783.62
 2019-09-20  10244.29       None  102442.90    768.32
 2019-09-27      None    8177.91  102442.90    613.34
 2019-11-22      None    7268.23   81779.10    545.12,
 'rsi_rsi_lower20':            buy_price sell_price       cost commision
 dt                                                  
 2018-12-16   3211.72       None   96351.60    722.64
 2019-04-03      None    4932.60   96351.60   1109.84
 2019-09-27   8063.73       None  129019.68    967.65,
 'rsi_rsi_lower30':            buy_price sell_price       cost commision
 dt                 